In [1]:
from scqbf.scqbf_instance import *
from scqbf.scqbf_solution import *
from scqbf.scqbf_evaluator import *
from scqbf.scqbf_ts import *

import pandas as pd
import numpy as np
import pickle

In [2]:
instance_paths = [(f"instances/gen{i}/instance{j}.txt", i, j) for i in range(1, 4) for j in range(1, 6)]
instance_paths

[('instances/gen1/instance1.txt', 1, 1),
 ('instances/gen1/instance2.txt', 1, 2),
 ('instances/gen1/instance3.txt', 1, 3),
 ('instances/gen1/instance4.txt', 1, 4),
 ('instances/gen1/instance5.txt', 1, 5),
 ('instances/gen2/instance1.txt', 2, 1),
 ('instances/gen2/instance2.txt', 2, 2),
 ('instances/gen2/instance3.txt', 2, 3),
 ('instances/gen2/instance4.txt', 2, 4),
 ('instances/gen2/instance5.txt', 2, 5),
 ('instances/gen3/instance1.txt', 3, 1),
 ('instances/gen3/instance2.txt', 3, 2),
 ('instances/gen3/instance3.txt', 3, 3),
 ('instances/gen3/instance4.txt', 3, 4),
 ('instances/gen3/instance5.txt', 3, 5)]

In [ ]:
def run_experiment(config: TSConfig) -> pd.DataFrame:    
    results = []
    exp_num = 1

    for instance_path, gen, inst in instance_paths:
        instance = read_max_sc_qbf_instance(instance_path)
        print(f"{exp_num}: {inst}th instance of generation strategy {gen}. Path: {instance_path}")
        exp_num += 1
        
        if config.ibr_component is not None:
            config.ibr_component.set_instance(instance)
        
        time_limit = 600*3
        ts = ScQbfTS(instance, config, time_limit_secs=time_limit, patience=2500)
        best_solution = ts.solve()
        
        evaluator = ScQbfEvaluator(instance)
        results.append({
            'gen': gen,
            'inst': inst,
            'n': instance.n,
            'stop_reason': ts.stop_reason,
            'best_objective': evaluator.evaluate_objfun(best_solution),
            'coverage': evaluator.evaluate_coverage(best_solution),
            'time_taken': ts.execution_time
        })
        
        last_result = results[-1]
        print(f"\tBest objective value: {last_result['best_objective']:.4f}")
        print(f"Selected elements: {best_solution.elements}")
        print(f"\tCoverage: {last_result['coverage']:.2%}")
        print(f"\tTime taken (secs): {last_result['time_taken']:.4f} s")
        print(f"\tStop reason: {last_result['stop_reason']}")
        print()
    
    df = pd.DataFrame(results)
    return df

In [4]:
# Tenure parameters

T1 = 8
T2 = 25

# Experimento 1 - PADRÃO
Busca Tabu com método de busca first-improving, “tabu tenure” igual a T1 , estratégia
tabu padrão.

In [11]:
config = TSConfig(tenure=T1, search_strategy='first')
results_df = run_experiment(config)

with open("pickles/results_config_1.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)

1: 1th instance of generation strategy 1. Path: instances/gen1/instance1.txt
	Best objective value: 4531.4600
Selected elements: [114, 69, 158, 48, 119, 196, 65, 28, 9, 11, 92, 159, 41, 178, 150, 25, 5, 98, 174, 157, 12, 125, 112, 76, 123, 40, 105, 91, 67, 147, 148, 54, 142, 111, 102, 161, 172, 171, 135, 43, 113, 36, 79, 71, 83, 129, 163, 99, 47, 126, 116, 136, 46, 60, 68, 124, 72, 173, 144, 115, 154, 146, 187, 166, 110, 87, 22, 80, 14, 97, 167, 164, 151, 137, 198, 85, 189, 23, 78, 179, 195, 192, 53, 89, 44, 38, 27, 175, 50, 133, 37, 15, 118, 153, 131, 160, 64, 127, 145, 177, 139, 109, 10, 94, 1, 197, 101, 182, 90, 32, 128, 169, 117, 59, 81]
	Coverage: 100.00%
	Time taken (secs): 305.5462 s
	Stop reason: patience_exceeded

2: 2th instance of generation strategy 1. Path: instances/gen1/instance2.txt
	Best objective value: 4617.2100
Selected elements: [161, 82, 64, 132, 10, 63, 184, 32, 44, 69, 152, 111, 122, 18, 49, 162, 121, 42, 17, 48, 124, 46, 73, 185, 159, 134, 29, 21, 58, 179, 197,

,gen,inst,n,stop_reason,best_objective,coverage,time_taken
0,1,1,200,patience_exceeded,4531.46,1.0,305.546157
1,1,2,200,patience_exceeded,4617.21,1.0,300.943455
2,1,3,50,patience_exceeded,642.03,1.0,5.650706
3,1,4,25,patience_exceeded,243.69,1.0,0.905617
4,1,5,100,patience_exceeded,1525.48,1.0,39.231419
5,2,1,50,patience_exceeded,562.74,1.0,5.270715
6,2,2,400,time_limit,14401.08,1.0,1800.121871
7,2,3,25,patience_exceeded,210.16,1.0,0.873660
8,2,4,400,time_limit,13440.01,1.0,1800.885643
9,2,5,100,patience_exceeded,1506.45,1.0,41.356264


# Experimento 2 - PADRÃO+BEST
Busca Tabu PADRÃO mas com método de busca best-improving.

In [12]:
config = TSConfig(tenure=T1, search_strategy='best')
results_df = run_experiment(config)

with open("pickles/results_config_2.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)

1: 1th instance of generation strategy 1. Path: instances/gen1/instance1.txt
	Best objective value: 4611.4700
Selected elements: [146, 28, 25, 144, 142, 126, 46, 158, 12, 92, 195, 36, 5, 9, 154, 37, 48, 71, 147, 157, 67, 114, 198, 187, 123, 178, 116, 14, 65, 148, 110, 179, 91, 87, 23, 11, 131, 50, 125, 69, 38, 97, 32, 68, 22, 27, 90, 139, 192, 78, 112, 145, 151, 47, 72, 173, 127, 102, 54, 189, 167, 129, 113, 172, 44, 136, 98, 80, 94, 159, 150, 60, 105, 163, 171, 99, 166, 160, 137, 76, 169, 1, 111, 86, 175, 41, 15, 161, 174, 197, 40, 53, 133, 135, 43, 89, 153, 118, 83, 8, 182, 3, 115, 196, 79, 64, 109, 51, 61, 117, 134, 85, 10, 73, 81, 128, 33, 45, 132, 13, 74]
	Coverage: 100.00%
	Time taken (secs): 335.5740 s
	Stop reason: patience_exceeded

2: 2th instance of generation strategy 1. Path: instances/gen1/instance2.txt
	Best objective value: 4668.0700
Selected elements: [191, 62, 185, 122, 154, 25, 58, 5, 161, 45, 19, 35, 82, 197, 57, 90, 21, 32, 42, 198, 162, 101, 63, 132, 184, 46, 177,

,gen,inst,n,stop_reason,best_objective,coverage,time_taken
0,1,1,200,patience_exceeded,4611.47,1.0,335.574025
1,1,2,200,patience_exceeded,4668.07,1.0,337.554777
2,1,3,50,patience_exceeded,645.62,1.0,5.641091
3,1,4,25,patience_exceeded,261.06,1.0,0.757915
4,1,5,100,patience_exceeded,1522.90,1.0,43.174249
5,2,1,50,patience_exceeded,575.87,1.0,5.808622
6,2,2,400,time_limit,14385.97,1.0,1801.316877
7,2,3,25,patience_exceeded,210.16,1.0,0.906542
8,2,4,400,time_limit,13530.16,1.0,1800.907347
9,2,5,100,patience_exceeded,1550.49,1.0,45.162396


# Experimento 3 - PADRÃO+TENURE
Busca Tabu PADRÃO mas com novo valor de “tabu tenure” igual a T2 .

In [13]:
config = TSConfig(tenure=T2, search_strategy='first')
results_df = run_experiment(config)

with open("pickles/results_config_3.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)

1: 1th instance of generation strategy 1. Path: instances/gen1/instance1.txt
	Best objective value: 4485.0100
Selected elements: [48, 47, 71, 123, 12, 72, 187, 154, 146, 116, 178, 40, 38, 28, 27, 36, 92, 97, 46, 25, 150, 81, 14, 157, 79, 142, 69, 163, 67, 91, 172, 68, 76, 113, 85, 159, 80, 189, 32, 37, 179, 167, 122, 110, 177, 136, 145, 15, 94, 175, 160, 129, 131, 174, 161, 44, 65, 98, 148, 135, 41, 23, 105, 99, 54, 87, 11, 166, 128, 133, 89, 9, 144, 153, 102, 190, 126, 17, 112, 22, 176, 5, 195, 173, 171, 101, 59, 147, 63, 114, 90, 74, 33, 198, 192, 86, 51, 78, 197, 60, 117, 34, 50, 151, 1, 137, 139, 43, 158, 169, 124, 196, 111, 125, 115, 26, 134, 53, 119, 118, 3, 182, 8, 83]
	Coverage: 100.00%
	Time taken (secs): 313.2225 s
	Stop reason: patience_exceeded

2: 2th instance of generation strategy 1. Path: instances/gen1/instance2.txt
	Best objective value: 4627.7400
Selected elements: [26, 41, 161, 71, 42, 68, 101, 132, 57, 106, 127, 64, 7, 84, 163, 78, 171, 187, 40, 197, 179, 141, 10, 

,gen,inst,n,stop_reason,best_objective,coverage,time_taken
0,1,1,200,patience_exceeded,4485.01,1.0,313.222521
1,1,2,200,patience_exceeded,4627.74,1.0,319.663765
2,1,3,50,patience_exceeded,600.73,1.0,5.771103
3,1,4,25,patience_exceeded,243.69,1.0,0.923060
4,1,5,100,patience_exceeded,1525.48,1.0,40.091573
5,2,1,50,patience_exceeded,575.87,1.0,5.730958
6,2,2,400,time_limit,14427.58,1.0,1800.949022
7,2,3,25,patience_exceeded,210.16,1.0,0.888114
8,2,4,400,time_limit,13350.38,1.0,1801.105843
9,2,5,100,patience_exceeded,1512.59,1.0,39.258160


# Experimento 4 - PADRÃO+METHOD1
Busca Tabu PADRÃO mas com estratégia tabu alternativa 1 (Probabilistic TS)

In [15]:
config = TSConfig(tenure=T1, search_strategy='first', probabilistic_ts=True, probabilistic_param=0.6)
results_df = run_experiment(config)

with open("pickles/results_config_3.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)

1: 1th instance of generation strategy 1. Path: instances/gen1/instance1.txt
	Best objective value: 4564.8700
Selected elements: [67, 5, 114, 112, 125, 113, 136, 195, 158, 179, 11, 65, 178, 147, 123, 110, 72, 91, 173, 12, 116, 102, 146, 126, 46, 38, 90, 60, 71, 23, 97, 187, 87, 37, 129, 154, 48, 28, 14, 131, 44, 172, 40, 142, 15, 25, 47, 151, 68, 54, 36, 98, 166, 75, 163, 32, 69, 111, 76, 9, 80, 27, 78, 50, 145, 175, 189, 150, 127, 105, 167, 139, 137, 92, 99, 161, 160, 1, 197, 89, 41, 94, 159, 148, 86, 133, 192, 157, 174, 79, 169, 198, 171, 53, 177, 74, 101, 176, 43, 132, 153, 144, 34, 8, 16, 119, 22, 3, 124, 182, 135, 164, 83, 115, 196, 118, 168, 64, 77, 59, 109]
	Coverage: 100.00%
	Time taken (secs): 173.0103 s
	Stop reason: patience_exceeded

2: 2th instance of generation strategy 1. Path: instances/gen1/instance2.txt


KeyboardInterrupt: 

# Experimento 5 - PADRÃO+METHOD2
Busca Tabu PADRÃO mas com estratégia tabu alternativa 2 (Intesification by Restart).

In [5]:
ibr_component = RestartIntensificationComponent(restart_patience=100, max_fixed_elements=5)
config = TSConfig(tenure=T1, search_strategy='first', ibr_component=ibr_component)
results_df = run_experiment(config)

with open("pickles/results_config_3.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)

1: 1th instance of generation strategy 1. Path: instances/gen1/instance1.txt


TypeError: 'NoneType' object is not subscriptable